### Загружаю библиотеки и таблицы

In [1]:
import pandas as pd
pd.options.display.float_format = '{0:.15g}'.format
import seaborn as sns
from tqdm import tqdm
import numpy as np

In [2]:
address_clust = pd.read_csv('c:/users/tyuio/address_clust.csv')
address_clust.head()

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1


In [3]:
address_stats = pd.read_csv('c:/users/tyuio/address_stats.csv')
address_stats.head()

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0,1800000
1,87134766,45919235,19162323,0,1071052
2,87134767,46529090,19162323,1000000,0
3,87134768,46529091,19162323,1821052,0
4,87154706,46529090,19166856,0,1000000


### Соединяю, заполняю налы

In [4]:
result = pd.merge(address_clust, address_stats, on='address_id', how = 'outer')

In [5]:
result = result.fillna(0)

In [6]:
result = result.set_index('id')

### Группирую для простоты работы

In [7]:
Trans_groups = result.groupby('transaction_id')

### Словарь для хранения сумм переводов

In [8]:
Dict = {'Кластер 0 отправил в кластер 1:': 0,
        'Кластер 1 отправил в кластер 0:': 0,
        'Кластер 2 отправил в кластер 0:': 0,
        'Кластер 0 отправил в кластер 2:': 0,
        'Кластер 1 отправил в кластер 2:': 0,
        'Кластер 2 отправил в кластер 1:': 0}

### Маленькая функция

In [9]:
def update_dictionary(d, key, value):
    d[key] += value

### Цикл для отлова переводов между кластерами

In [10]:
for i in tqdm(Trans_groups.groups.keys()):
    x = Trans_groups.get_group(i).sort_values('id')
    for i in range(x.shape[0]):
        if x.iloc[i]['sent'] != 0:
            flag_sender = int(x.iloc[i]['cluster_id'])
        else:
            continue
        break
    summ_sent = 0
    summ_received = 0
    flag_receiver = np.nan
    summ_received_inside_cluster = 0
    for i in range(x.shape[0]):
        if x.iloc[i]['sent'] != 0 and x.iloc[i]['cluster_id'] == flag_sender:
            summ_sent += x.iloc[i]['sent']
        elif x.iloc[i]['received'] != 0 and x.iloc[i]['cluster_id'] != flag_sender or x.iloc[i]['cluster_id'] == flag_receiver:
            summ_received = x.iloc[i]['received']
            flag_receiver = x.iloc[i]['cluster_id']
            if flag_receiver == 1 and flag_sender == 0:
                update_dictionary(Dict, 'Кластер 0 отправил в кластер 1:', summ_received/10**8)
            elif flag_receiver == 0 and flag_sender == 1:
                update_dictionary(Dict, 'Кластер 1 отправил в кластер 0:', summ_received/10**8)
            elif flag_receiver == 2 and flag_sender == 1:
                update_dictionary(Dict, 'Кластер 1 отправил в кластер 2:', summ_received/10**8)
            elif flag_receiver == 1 and flag_sender == 2:
                update_dictionary(Dict, 'Кластер 2 отправил в кластер 1:', summ_received/10**8)
            elif flag_receiver == 2 and flag_sender == 0:
                update_dictionary(Dict, 'Кластер 0 отправил в кластер 2:', summ_received/10**8)
            elif flag_receiver == 0 and flag_sender == 2:
                    update_dictionary(Dict, 'Кластер 2 отправил в кластер 0:', summ_received/10**8)                
        elif x.iloc[i]['received'] != 0 and flag_sender == x.iloc[i]['cluster_id']:
            summ_received_inside_cluster += x.iloc[i]['received']


100%|███████████████████████████████████████████████████████████████████████████| 13753/13753 [02:12<00:00, 103.87it/s]


# Новые данные

In [11]:
Dict

{'Кластер 0 отправил в кластер 1:': 297.1196838799924,
 'Кластер 1 отправил в кластер 0:': 300.55392887999847,
 'Кластер 2 отправил в кластер 0:': 4.869840000000003,
 'Кластер 0 отправил в кластер 2:': 9.740138609999246,
 'Кластер 1 отправил в кластер 2:': 129.2432399999996,
 'Кластер 2 отправил в кластер 1:': 135.82000000000397}

### Некорректные данные (старые)
Кластер 0 отправил в кластер 1: 292.98638548 биткоинов<br />
Кластер 1 отправил в кластер 0: 325.3799055 биткоинов<br />
Кластер 2 отправил в кластер 0: 3.58116 биткоинов<br />
Кластер 0 отправил в кластер 2: 13.87343701 биткоинов<br />
Кластер 1 отправил в кластер 2: 104.41726338 биткоинов<br />
Кластер 2 отправил в кластер 1: 137.10868 биткоинов<br />

### Размер fee в биткоинах общая неправильная

<img src="https://cdn.shopify.com/s/files/1/1746/9141/products/Under_Construction_Tape.png?v=1557281843" alt="Italian Trulli">

In [12]:
fee = (Trans_groups.agg('sum')['received'] - Trans_groups.agg('sum')['sent']).abs()

In [13]:
fee.sum()/10**8

2.36441506